In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O D:\Projects\NLP\dataset\bbc-text.csv


SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\Gow/etc/wgetrc
--2020-07-06 02:24:03--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com... 172.217.0.240, 172.217.165.16, 172.217.164.240, ...
Connecting to storage.googleapis.com|172.217.0.240|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: `D:/Projects/NLP/dataset/bbc-text.csv'

     0K .......... .......... .......... .......... ..........  1% 1.11M 4s
    50K .......... .......... .......... .......... ..........  2% 1.62M 4s
   100K .......... .......... .......... .......... ..........  3% 1.37M 4s
   150K .......... .......... .......... .......... ..........  4% 5.98M 3s
   200K .......... .......... .......... .......... ..........  5% 1.63M 3s
   250K .......... .......... .......... .......... ..........  6% 

In [2]:
import csv
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

  1800K .......... .......... .......... .......... .......... 37% 4.99M 2s
  1850K .......... .......... .......... .......... .......... 38% 8.98M 2s
  1900K .......... .......... .......... .......... .......... 39% 7.34M 1s
  1950K .......... .......... .......... .......... .......... 40% 2.18M 1s
  2000K .......... .......... .......... .......... .......... 41% 1.64M 1s
  2050K .......... .......... .......... .......... .......... 42% 9.18M 1s
  2100K .......... .......... .......... .......... .......... 43% 1.57M 1s
  2150K .......... .......... .......... .......... .......... 44% 1.67M 1s
  2200K .......... .......... .......... .......... .......... 45% 2.68M 1s
  2250K .......... .......... .......... .......... .......... 46% 9.11M 1s
  2300K .......... .......... .......... .......... .......... 47% 1.66M 1s
  2350K .......... .......... .......... .......... .......... 48% 2.41M 1s
  2400K .......... .......... .......... .......... .......... 49% 2.61M 1s
  2450K ....

In [3]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could",
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has",
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him",
             "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it",
             "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once",
             "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she",
             "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their",
             "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're",
             "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd",
             "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while",
             "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your",
             "yours", "yourself", "yourselves" ];

In [18]:
sentences = []
labels = []

with open('dataset/bbc-text.csv' , 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader,None)
    for row in reader:
        labels.append(row[0])       
        sentence = row[1]
        
        for word in stopwords:
            token = " {} ".format(word)
            sentence = sentence.replace(token, " ")
            sentence = sentence.replace("  ", " ")
        
        sentences.append(sentence)

print(len(sentences))
print(sentences[0])

2225
tv future hands viewers home theatre systems plasma high-definition tvs digital video recorders moving living room way people watch tv will radically different five years time. according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend programmes content will delivered viewers via home networks cable satellite telecoms companies broadband service providers front rooms portable devices. one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes like us s tivo uk s sky+ system allow people record store play pause forward wind tv programmes want. essentially technology allows much personalised tv. also built-in high-definition tv sets big business japan us slower take off europe lack high-definition programming. not can people forward wind adverts can also forget abiding network channel schedules putting together a-la-carte entertainment. us networks cable sa

In [19]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

print(len(word_index))

29714


In [20]:
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post')
print(padded[0])
print(padded.shape)

[  96  176 1158 ...    0    0    0]
(2225, 2442)


In [22]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
label_word_index = label_tokenizer.word_index
label_seq = label_tokenizer.texts_to_sequences(labels)
#print(label_seq)
print(label_word_index)

{'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}
